In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# reset defalult plotting values
plt.rcParams['figure.figsize'] = (10, 7)
plt.rc('font', family='sans-serif')
plt.rc('axes', labelsize=14)
plt.rc('axes', labelweight='bold')
plt.rc('axes', titlesize=16)
plt.rc('axes', titleweight='bold')
plt.rc('axes', linewidth=2)

# SQL
## The language of relational databases

![](https://wiki.postgresql.org/images/3/30/PostgreSQL_logo.3colors.120x120.png)


### Prof. Robert Quimby
&copy; 2019 Robert Quimby

## In this tutorial you will...

- hear about the role of databases
- learn how to retrieve records (data) from a database
- run basic SQL queries on an example database
- use ADQL to query the Gaia database

## Databases

![](http://www.sqlitetutorial.net/wp-content/uploads/2015/12/RDBMS-Client-Server-Architecture.jpg)

## Structured Query Language (SQL)

## Resources
  - [SQL tutorial](https://www.w3schools.com/sql/default.asp) from w3schools.com

#### Basic grammar for SQL queries

- **SELECT** [comma separated list of values to select] 
  - **FROM** [names of the tables to select values from]
  - **WHERE** [conditions on what values to select]
  - **ORDER BY** [options to sort the selected values]
  - **LIMIT** [option to limit the number of records retrieved]


## SQLite

https://www.sqlite.org/index.html

![](http://www.sqlitetutorial.net/wp-content/uploads/2015/12/What-is-SQLite.jpg)



## SQLite database queries in python

In [ ]:
# connect to a database
import sqlite3
conn = sqlite3.connect(????)

# data science package to work with query results
import pandas as pd

## Schema of tables in the example database ('example.db')

The **stars** table holds data for 9110 stars in the Yale Bright Star Catalog. The table has the following columns:
- **id** - (integer) a unique number for each star in the table
- **name** - (text) name of the star (if any)
- **ra** - (real) right ascension in degrees 
- **dec** - (real) declination in degrees 
- **vmag** - (real) V-band magnitude of the star
- **sp_type** - (text) spectral type of the star

The **spec_class** table holds data for (some of the) different spectral types including:
- **sp_class** - Morgan–Keenan (Yerkes) spectral class (OBAFGKM)
- **lum_class** - luminosity class (V=Main Sequence; III=Giant; I=Super Giant)
- **temperature** - typical photospheric temperature
- **abs_mag** - typical absolute magnitude

## SELECTing data from a database

In [ ]:
# load everything from the stars table
pd.read_sql_query(????, conn)

In [ ]:
# load just the ra and dec of stars in the stars table
pd.read_sql_query("SELECT ???? FROM stars", conn)

In [ ]:
# load everything from the spec_class table
pd.read_sql_query("SELECT * FROM spec_class", conn)

## Using the WHERE clause

In [ ]:
pd.read_sql_query("SELECT * FROM stars WHERE ????", conn)

## Combining conditions with AND

In [ ]:
query = """
SELECT ra, dec
FROM stars
WHERE vmag < 2.5
  AND ra > 65
  AND ra < 95
  AND dec > -10
  AND dec < 10
"""
stars = pd.read_sql_query(query, conn)

In [ ]:
# plot the R.A., Dec. of the selected stars
plt.axes(aspect='equal')
plt.plot(stars['ra'], stars['dec'], 'ro')
plt.gca().invert_xaxis()

In [ ]:
# select by luminosity class
query = "SELECT * FROM spec_class WHERE lum_class='{}'"
ms = pd.read_sql_query(query.format('V'), conn)
g = pd.read_sql_query(query.format('III'), conn)
sg = pd.read_sql_query(query.format('I'), conn)

In [ ]:
# plot an HR diagram
plt.plot(ms['temperature'], ms['abs_mag'], 'go')
plt.plot(g['temperature'], g['abs_mag'], 'ro')
plt.plot(sg['temperature'], sg['abs_mag'], 'bo')
plt.gca().invert_xaxis()
plt.gca().invert_yaxis()
plt.xscale('log')
plt.xlabel('Temperature (K)')
plt.ylabel('Absolute V-band Magnitude');

## Computing values in queries

In [ ]:
# use SQL as a calculator
pd.read_sql_query("SELECT ????", conn)

In [ ]:
# retrieve computed values
pd.read_sql_query("SELECT ???? FROM stars ", conn)

## Joining tables

In [ ]:
query = """
SELECT name, ra, dec, vmag, temperature, abs_mag
FROM stars, spec_class
WHERE 
  ????
"""
pd.read_sql_query(query, conn)

## Relation between observed mag, absolute mag, and distance

$$D = 10^{ (m - M + 5)/5 }$$

## Adding functions

In [ ]:
# raising a number to some power in numpy
????

In [ ]:
# add the POWER function to queries
conn.create_function("power", ????, ????)

In [ ]:
query = """
SELECT name, ra, dec, vmag, abs_mag, POWER(10, (vmag - abs_mag + 5) / 5) AS dist
FROM stars, spec_class
WHERE 
  sp_type = sp_class || lum_class
ORDER BY dist 
LIMIT 10
"""
pd.read_sql_query(query, conn)

## Astronomical Data Query Language (ADQL)

Based on SQL but designed for astronomers

[Gaia query web interface](http://gea.esac.esa.int/archive/)
- [Basic ADQL syntax](https://gea.esac.esa.int/archive-help/adql/index.html) from Gaia web pages
- [Example ADQL queries](https://gea.esac.esa.int/archive-help/adql/examples/index.html) from Gaia

In [ ]:
from astroquery.gaia import Gaia

In [ ]:
# query Gaia database to select stars in the Gaia DR2 catalog 
# near (R.A, Dec.) = (0, 0)
# near the North Celestial Pole

query = """SELECT ra, dec
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
  POINT('ICRS', ra, dec),
  CIRCLE('ICRS', 0, 0, 0.25))
  AND phot_bp_mean_mag < 15
"""
job = Gaia.launch_job_async(query)

In [ ]:
# get the search results
gaia = job.get_results()
gaia